In [1]:
import pandas as pd

df_fit = pd.read_csv('../spam_text_data/bg_message_vector.csv')
df = pd.read_csv('../spam_text_data/test_message_vector.csv')
df

,00,000,10,11,110,112679,114998,12,120,123,...,www,xyz,year,yes,you,your,yours,zelle,zen,zpass
0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.408541,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.109489,0.0,0.0,0.0,0.0
1,0.0,0.0,0.253844,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.259553,0.0,0.453305,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.076804,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
109,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.114350,0.0,0.0,0.0,0.0
110,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
111,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.074505,0.0,0.0,0.0,0.0


In [2]:
df_text = pd.read_csv('../spam_text_data/test_all.csv')
df_text

,scam_flag,message
0,0,reminder : your netflix account payment of $ 1...
1,0,electricity bill due april 10 . amount : $ 125...
2,0,"final notice : electricity bill due tomorrow ,..."
3,0,"friendly reminder : your rent payment of $ 1,2..."
4,0,gym renewal reminder : pay $ 50 before april 1...
...,...,...
108,1,internet service termination warning click lin...
109,1,final warning your insurance is invalid pay at...
110,1,account blocked reactivate now by paying at n...
111,1,pay your toll pass fee now to avoid fines clic...


In [3]:
from sklearn.cluster import KMeans
import time

n_clusters = 2

kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
kmeans.fit(df_fit)

start_time = time.time()
labels = kmeans.predict(df)
end_time = time.time()
kmeans_time = end_time - start_time

In [4]:
import numpy as np
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import confusion_matrix


def match_cluster_labels(true_labels, kmeans_labels):
    """
    matching cluster labels to achieve comparable cluster assignments.
    """
    conf_matrix = confusion_matrix(true_labels, kmeans_labels)

    row_ind, col_ind = linear_sum_assignment(-conf_matrix)

    label_mapping = {old: new for old, new in zip(col_ind, row_ind)}

    new_kmeans_labels = np.array([label_mapping[label] for label in kmeans_labels])

    return new_kmeans_labels

# Usage
corrected_labels = match_cluster_labels(df_text["scam_flag"], labels)


In [5]:
from sklearn.metrics import silhouette_score, adjusted_rand_score, normalized_mutual_info_score

nmi = normalized_mutual_info_score(df_text["scam_flag"], corrected_labels)
print(f"Normalized Mutual Information: {nmi}")
ari = adjusted_rand_score(df_text["scam_flag"], corrected_labels)
print(f"Adjusted Rand Index: {ari}")
silhouette_score = silhouette_score(df, corrected_labels)
print(f"Silhouette Score: {silhouette_score}")

Normalized Mutual Information: 0.12240514113889026
Adjusted Rand Index: 0.02566711140760507
Silhouette Score: 0.023833296952450587


In [6]:
print("Total Time:", kmeans_time, " seconds")

Total Time: 0.002646923065185547  seconds
